In [61]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense

ImportError: No module named 'matplotlib'

In [21]:
# inputs
trends_x = ['Google.csv', 'battery.csv', 'solar.csv']

# label
trend_y = 'Tesla.csv'

In [22]:
x = []
for each in trends_x:
    x.append(pd.read_csv(each))

In [32]:
data = np.array(x[0].merge(x[1], on='Date').merge(x[2], on='Date'))
# date, google, battery, solar, tesla

label = np.array(pd.read_csv(trend_y))

In [57]:
train_x = data[:330,:]
train_y = label[:330,:]
test_x = data[330:,:]
test_y = label[330:,:]

In [51]:
model = Sequential()
model.add(Dense(8, input_shape=(3,), activation='relu'))
model.add(Dense(8, input_shape=(3,), activation='relu'))
model.add(Dense(8, input_shape=(3,), activation='relu'))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

In [52]:
model.fit(train_x[:,1:], train_y[:,1:], epochs=20, verbose=1)

Epoch 1/20
330/330 [==============================] - 0s - loss: 197.3607 - acc: 0.0000e+00     
Epoch 2/20
330/330 [==============================] - 0s - loss: 114.9133 - acc: 0.0030         
Epoch 3/20
330/330 [==============================] - 0s - loss: 80.4653 - acc: 0.0000e+00     
Epoch 4/20
330/330 [==============================] - 0s - loss: 71.0179 - acc: 0.0030          
Epoch 5/20
330/330 [==============================] - 0s - loss: 67.5674 - acc: 0.0061          
Epoch 6/20
330/330 [==============================] - 0s - loss: 63.7213 - acc: 0.0061      

In [53]:
trainScore = model.evaluate(train_x[:,1:], train_y[:,1:], verbose=1)

 32/330 [=>............................] - ETA: 0s

In [59]:
testScore = model.evaluate(test_x[:,1:], test_y[:,1:], verbose=1)

 32/140 [=====>........................] - ETA: 0s

In [68]:
look_back = 10 # number of previous time steps to use as input to predict the next time period
trainPredict = model.predict(train_x[:,1:])
testPredict = model.predict(test_x[:,1:])

# shift train predictions for plotting
trainPredictPlot = np.empty_like(data[:,1:])
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict
 
# shift test predictions for plotting
testPredictPlot = np.empty_like(data[:,1:])
testPredictPlot[:, :] = np.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(data[:,1:])-1, :] = testPredict

ValueError: could not broadcast input array from shape (140,1) into shape (118,3)